In [1]:
import numpy as np
import pandas as pd

import pymysql
import dotenv
import os
import pickle
import seaborn as sns
import matplotlib

from datetime import timedelta

import matplotlib.pyplot as plt

# Normalization/Standardization
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder

from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, Lasso
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, Dropout, Conv1D, GRU
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.saving import save_model

# 경고 무시 코드 추가
import warnings
warnings.filterwarnings('ignore')

def pltconfig_default() :
  sns.reset_defaults()
  %matplotlib inline

pltconfig_default()

matplotlib.rcParams

matplotlib.rcParams['font.family']

current_font_list = matplotlib.rcParams['font.family']

font_path = 'C:\\Windows\\Fonts\\batang.ttc'

kfont = matplotlib.font_manager.FontProperties(fname=font_path).get_name()

matplotlib.rcParams['font.family'] = [kfont] + current_font_list

In [2]:
St_NotEncode_data = pd.read_pickle("StandardScalar_final_data")

print(St_NotEncode_data.shape)

(17422, 9)


In [3]:
# Feature와 Label 분리하기
def Feature_Label(datafile) :
    X = datafile.iloc[:,:-1]
    y = datafile.iloc[:,-1]

    return X, y

print("StandardScaler Not Encode Data")
SNE_X, SNE_y = Feature_Label(St_NotEncode_data)
print(SNE_X.shape, SNE_y.shape)
SNE_X_train, SNE_X_test, SNE_y_train, SNE_y_test = train_test_split(SNE_X, SNE_y, test_size=0.2, random_state=10, shuffle=False)
print(SNE_X_train.shape, SNE_X_test.shape, SNE_y_train.shape, SNE_y_test.shape)
print("=======================================")


WINDOW_SIZE=3
BATCH_SIZE=28

StandardScaler Not Encode Data
(17422, 8) (17422,)
(13937, 8) (3485, 8) (13937,) (3485,)


In [4]:
# X 값 Window Dataset 구성
ds_x = tf.data.Dataset.from_tensor_slices(SNE_X_train).window(WINDOW_SIZE, stride=1, shift=1, drop_remainder=True)  # 여기서는 X값만 별도로 구성하므로 WINDOW_SIZE 값에 1을 더하지 않는다!
ds_x = ds_x.flat_map(lambda x : x.batch(WINDOW_SIZE))

In [5]:
# y 값 Window Dataset 구성
ds_y = tf.data.Dataset.from_tensor_slices(SNE_y_train[WINDOW_SIZE:])

In [6]:
train_data = tf.data.Dataset.zip((ds_x, ds_y)).batch(BATCH_SIZE)

In [7]:
for x, y in train_data.take(1) :
  print(x[:3])
  print()
  print(y[:3])

tf.Tensor(
[[[-0.11352868 -0.09386187 -0.6825146   0.09445245 -0.66581843
   -1.23511514 -0.21825161 -1.53935952]
  [-0.11352868 -0.9225242  -1.06038321  0.4051263  -0.66581843
   -1.23511514 -0.21825161 -1.57681737]
  [-0.11352868 -1.02610699 -1.06038321  0.53827224 -0.66581843
   -1.23511514 -0.21825161 -1.62925834]]

 [[-0.11352868 -0.9225242  -1.06038321  0.4051263  -0.66581843
   -1.23511514 -0.21825161 -1.57681737]
  [-0.11352868 -1.02610699 -1.06038321  0.53827224 -0.66581843
   -1.23511514 -0.21825161 -1.62925834]
  [-0.11352868 -1.12968978 -1.06038321  0.71580015 -0.66581843
   -1.23511514 -0.21825161 -1.65922462]]

 [[-0.11352868 -1.02610699 -1.06038321  0.53827224 -0.66581843
   -1.23511514 -0.21825161 -1.62925834]
  [-0.11352868 -1.12968978 -1.06038321  0.71580015 -0.66581843
   -1.23511514 -0.21825161 -1.65922462]
  [-0.11352868 -0.9225242  -1.06038321  0.80456411 -0.66581843
   -1.23511514 -0.21825161 -1.68919089]]], shape=(3, 3, 8), dtype=float64)

tf.Tensor([0. 0. 0.], 

In [8]:
pd.concat([SNE_X_train, SNE_y_train], axis=1).head(10)

,0,1,2,3,4,5,6,7,8
0,-0.113529,-0.093862,-0.682515,0.094452,-0.665818,-1.235115,-0.218252,-1.539360,0.0
1,-0.113529,-0.922524,-1.060383,0.405126,-0.665818,-1.235115,-0.218252,-1.576817,0.0
2,-0.113529,-1.026107,-1.060383,0.538272,-0.665818,-1.235115,-0.218252,-1.629258,0.0
3,-0.113529,-1.129690,-1.060383,0.715800,-0.665818,-1.235115,-0.218252,-1.659225,0.0
4,-0.113529,-0.922524,-1.060383,0.804564,-0.665818,-1.235115,-0.218252,-1.689191,0.0
5,-0.113529,-1.026107,-1.060383,0.937710,-0.665818,-1.235115,-0.218252,-1.719157,0.0
6,-0.113529,-0.922524,-1.060383,0.893328,-0.665818,-1.235115,-0.218252,-1.734140,0.0
7,-0.113529,-0.922524,-1.060383,0.982092,0.549517,-1.235115,-0.333181,-1.741632,6000.0
8,-0.113529,-1.129690,-1.060383,0.538272,1.764852,0.769233,-0.226741,-1.636750,16000.0
9,-0.113529,-0.922524,-1.060383,0.005688,1.764852,-0.984572,-0.322080,-1.329596,24000.0


In [9]:
# 위 내용을 함수 형태로 만들고 싶은 경우

def windowed_dataset(x, y, window_size, batch_size, shuffle) :
  ds_x = tf.data.Dataset.from_tensor_slices(SNE_X_train).window(window_size, stride=1, shift=1, drop_remainder=True) 
  ds_x = ds_x.flat_map(lambda x : x.batch(window_size))
  
  ds_y = tf.data.Dataset.from_tensor_slices(SNE_y_train[window_size:])
  
  ds = tf.data.Dataset.zip((ds_x, ds_y))
  
  if shuffle:
    ds = ds.shuffle(1000)
  
  return ds.batch(batch_size).prefetch(1)

In [10]:
train_data = windowed_dataset(SNE_X_train, SNE_y_train, WINDOW_SIZE, BATCH_SIZE, True)
train_data_ns = windowed_dataset(SNE_X_train, SNE_y_train, WINDOW_SIZE, BATCH_SIZE, False)
test_data = windowed_dataset(SNE_X_test, SNE_y_test, WINDOW_SIZE, BATCH_SIZE, False)

In [11]:
for data in train_data.take(1) :
  print(f'{data[0].shape}, {data[1].shape}')

(28, 3, 8), (28,)


In [12]:
lstm_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal',
         activation='relu',
         input_shape=[WINDOW_SIZE, 8]),
  # LSTM(128, activation='tanh', return_sequences=True),  
  LSTM(64, activation='tanh', return_sequences=True),
  LSTM(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [13]:
rnn_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal', 
         activation='relu',
         input_shape=[WINDOW_SIZE, 8]),
  # SimpleRNN(128, activation='tanh', return_sequences=True),  
  SimpleRNN(64, activation='tanh', return_sequences=True),
  SimpleRNN(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [14]:
gru_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal', 
         activation='relu',
         input_shape=[WINDOW_SIZE, 8]),
  # GRU(128, activation='tanh', return_sequences=True),  
  GRU(64, activation='tanh', return_sequences=True),
  GRU(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [15]:
loss = Huber() 
optimizer = Adam(learning_rate=0.001)
lstm_model.compile(loss=loss, optimizer=optimizer, metrics=['mse', 'mae'])
rnn_model.compile(loss=loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse', 'mae'])
gru_model.compile(loss=loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse', 'mae'])

In [16]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10)
       
es = EarlyStopping(monitor='val_loss', patience=20)

mc_lstm = ModelCheckpoint('./Control_Weights/low_neuron_group/new_data/new_multi_stne_lstm_weight_neuron.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_lstm_ns = ModelCheckpoint('./Control_Weights/low_neuron_group/new_data/new_multi_stne_lstm_weight_ns_neuron.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_rnn = ModelCheckpoint('./Control_Weights/low_neuron_group/new_data/new_multi_stne_rnn_weight_neuron.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_rnn_ns = ModelCheckpoint('./Control_Weights/low_neuron_group/new_data/new_multi_stne_rnn_weight_ns_neuron.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_gru = ModelCheckpoint('./Control_Weights/low_neuron_group/new_data/new_multi_stne_gru_weight_neuron.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_gru_ns = ModelCheckpoint('./Control_Weights/low_neuron_group/new_data/new_multi_stne_gru_weight_ns_neuron.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)

In [17]:
lstm_history = lstm_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_lstm])
lstm_history_ns = lstm_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_lstm_ns])

Epoch 1/500
    494/Unknown - 11s 6ms/step - loss: 7632.2896 - mse: 183839136.0000 - mae: 7632.6719
Epoch 1: val_loss improved from inf to 7608.69775, saving model to ./Control_Weights/low_neuron_group/new_data\new_multi_stne_lstm_weight_neuron.h5
498/498 [==============================] - 18s 19ms/step - loss: 7621.0840 - mse: 183618160.0000 - mae: 7621.4663 - val_loss: 7608.6978 - val_mse: 183079584.0000 - val_mae: 7609.1172 - lr: 0.0010
Epoch 2/500
492/498 [============================>.] - ETA: 0s - loss: 7589.1294 - mse: 182338976.0000 - mae: 7589.4897
Epoch 2: val_loss improved from 7608.69775 to 7562.90039, saving model to ./Control_Weights/low_neuron_group/new_data\new_multi_stne_lstm_weight_neuron.h5
498/498 [==============================] - 4s 7ms/step - loss: 7586.8071 - mse: 182193504.0000 - mae: 7587.1665 - val_loss: 7562.9004 - val_mse: 181167008.0000 - val_mae: 7563.2427 - lr: 0.0010
Epoch 3/500
497/498 [============================>.] - ETA: 0s - loss: 7522.9790 - mse:

In [18]:
lstm_save_path = f"./Control_Models/low_neuron_group/new_data/lstm_model"

save_model(lstm_model, lstm_save_path, overwrite=True)

INFO:tensorflow:Assets written to: ./Control_Models/low_neuron_group/new_data/lstm_model\assets


INFO:tensorflow:Assets written to: ./Control_Models/low_neuron_group/new_data/lstm_model\assets


In [19]:
rnn_history = rnn_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_rnn])
rnn_history_ns = rnn_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_rnn_ns])

Epoch 1/500
    484/Unknown - 3s 2ms/step - loss: 7596.6997 - mse: 183268944.0000 - mae: 7597.0781
Epoch 1: val_loss improved from inf to 7620.44092, saving model to ./Control_Weights/low_neuron_group/new_data\new_multi_stne_rnn_weight_neuron.h5
498/498 [==============================] - 5s 5ms/step - loss: 7626.3096 - mse: 183861984.0000 - mae: 7626.6885 - val_loss: 7620.4409 - val_mse: 183627792.0000 - val_mae: 7620.7646 - lr: 0.0010
Epoch 2/500
474/498 [===========================>..] - ETA: 0s - loss: 7587.3521 - mse: 183144640.0000 - mae: 7587.6992
Epoch 2: val_loss improved from 7620.44092 to 7599.11963, saving model to ./Control_Weights/low_neuron_group/new_data\new_multi_stne_rnn_weight_neuron.h5
498/498 [==============================] - 2s 4ms/step - loss: 7610.5410 - mse: 183216160.0000 - mae: 7610.8921 - val_loss: 7599.1196 - val_mse: 182716736.0000 - val_mae: 7599.5239 - lr: 0.0010
Epoch 3/500
479/498 [===========================>..] - ETA: 0s - loss: 7584.5015 - mse: 1824

In [20]:
rnn_save_path = f"./Control_Models/low_neuron_group/new_data/rnn_model"

save_model(rnn_model, rnn_save_path, overwrite=True)

INFO:tensorflow:Assets written to: ./Control_Models/low_neuron_group/new_data/rnn_model\assets


INFO:tensorflow:Assets written to: ./Control_Models/low_neuron_group/new_data/rnn_model\assets


In [21]:
gru_history = gru_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_gru])
gru_history_ns = gru_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_gru_ns])

Epoch 1/500
    497/Unknown - 5s 3ms/step - loss: 7617.2676 - mse: 183490688.0000 - mae: 7617.6572
Epoch 1: val_loss improved from inf to 7602.65039, saving model to ./Control_Weights/low_neuron_group/new_data\new_multi_stne_gru_weight_neuron.h5
498/498 [==============================] - 7s 6ms/step - loss: 7619.7314 - mse: 183575504.0000 - mae: 7620.1211 - val_loss: 7602.6504 - val_mse: 182860752.0000 - val_mae: 7603.0181 - lr: 0.0010
Epoch 2/500
487/498 [============================>.] - ETA: 0s - loss: 7553.9146 - mse: 181334768.0000 - mae: 7554.2627
Epoch 2: val_loss improved from 7602.65039 to 7536.35938, saving model to ./Control_Weights/low_neuron_group/new_data\new_multi_stne_gru_weight_neuron.h5
498/498 [==============================] - 2s 5ms/step - loss: 7573.6323 - mse: 181670800.0000 - mae: 7573.9819 - val_loss: 7536.3594 - val_mse: 180257504.0000 - val_mae: 7536.7383 - lr: 0.0010
Epoch 3/500
484/498 [============================>.] - ETA: 0s - loss: 7490.3125 - mse: 1789

In [22]:
gru_save_path = f"./Control_Models/low_neuron_group/new_data/gru_model"

save_model(gru_model, gru_save_path, overwrite=True)

INFO:tensorflow:Assets written to: ./Control_Models/low_neuron_group/new_data/gru_model\assets


INFO:tensorflow:Assets written to: ./Control_Models/low_neuron_group/new_data/gru_model\assets


In [23]:
# 종합 결과

history_list = ["lstm_history", "rnn_history", "gru_history", "lstm_history_ns", "rnn_history_ns", "gru_history_ns"]
def result(historys) :
  for name, history in globals().items() :
    if name in history_list :
      print(f"-------------{name}-------------")
      val_loss = min(history.history['val_loss'])
      val_mse = min(history.history['val_mse'])
      val_mae = min(history.history['val_mae'])
      print(f"{name} Validation Loss:", val_loss)
      print(f"{name} Validation MSE:", val_mse)
      print(f"{name} Validation MAE:", val_mae)

result(history_list)

-------------lstm_history-------------
lstm_history Validation Loss: 353.0738525390625
lstm_history Validation MSE: 1752314.125
lstm_history Validation MAE: 353.3738098144531
-------------lstm_history_ns-------------
lstm_history_ns Validation Loss: 359.65142822265625
lstm_history_ns Validation MSE: 1750014.0
lstm_history_ns Validation MAE: 359.9536437988281
-------------rnn_history-------------
rnn_history Validation Loss: 774.947509765625
rnn_history Validation MSE: 4368276.0
rnn_history Validation MAE: 775.2745971679688
-------------rnn_history_ns-------------
rnn_history_ns Validation Loss: 717.7471923828125
rnn_history_ns Validation MSE: 4210283.0
rnn_history_ns Validation MAE: 718.0726928710938
-------------gru_history-------------
gru_history Validation Loss: 455.72772216796875
gru_history Validation MSE: 2342793.75
gru_history Validation MAE: 456.04608154296875
-------------gru_history_ns-------------
gru_history_ns Validation Loss: 460.4094543457031
gru_history_ns Validation M

In [24]:
St_NotEncode_Basic_data = pd.read_pickle("Basic_StandardScalar_final_data")
  
print(St_NotEncode_Basic_data.shape)

(52578, 9)


In [25]:
# Feature와 Label 분리하기
def Feature_Label(datafile) :
    X = datafile.iloc[:,:-1]
    y = datafile.iloc[:,-1]

    return X, y

print("StandardScaler Not Encode Data")
SB_X, SB_y = Feature_Label(St_NotEncode_Basic_data)
print(SNE_X.shape, SNE_y.shape)
SB_X_train, SB_X_test, SB_y_train, SB_y_test = train_test_split(SB_X, SB_y, test_size=0.2, random_state=10, shuffle=False)
print(SB_X_train.shape, SB_X_test.shape, SB_y_train.shape, SB_y_test.shape)
print("=======================================")


WINDOW_SIZE=3
BATCH_SIZE=28

StandardScaler Not Encode Data
(17422, 8) (17422,)
(42062, 8) (10516, 8) (42062,) (10516,)


In [26]:
def windowed_dataset(x, y, window_size, batch_size, shuffle) :
  ds_x = tf.data.Dataset.from_tensor_slices(x).window(window_size, stride=1, shift=1, drop_remainder=True) 
  ds_x = ds_x.flat_map(lambda x : x.batch(window_size))
  
  ds_y = tf.data.Dataset.from_tensor_slices(y[window_size:])
  
  ds = tf.data.Dataset.zip((ds_x, ds_y))
  
  if shuffle:
    ds = ds.shuffle(1000)
  
  return ds.batch(batch_size).prefetch(1)

In [27]:
train_data = windowed_dataset(SB_X_train, SB_y_train, WINDOW_SIZE, BATCH_SIZE, True)
train_data_ns = windowed_dataset(SB_X_train, SB_y_train, WINDOW_SIZE, BATCH_SIZE, False)
test_data = windowed_dataset(SB_X_test, SB_y_test, WINDOW_SIZE, BATCH_SIZE, False)

In [28]:
for data in train_data.take(1) :
  print(f'{data[0].shape}, {data[1].shape}')

(28, 3, 8), (28,)


In [29]:
lstm_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal',
         activation='relu',
         input_shape=[WINDOW_SIZE, 8]),
  # LSTM(128, activation='tanh', return_sequences=True),  
  LSTM(64, activation='tanh', return_sequences=True),
  LSTM(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [30]:
rnn_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal', 
         activation='relu',
         input_shape=[WINDOW_SIZE, 8]),
  # SimpleRNN(128, activation='tanh', return_sequences=True),  
  SimpleRNN(64, activation='tanh', return_sequences=True),
  SimpleRNN(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [31]:
gru_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal', 
         activation='relu',
         input_shape=[WINDOW_SIZE, 8]),
  # GRU(128, activation='tanh', return_sequences=True),  
  GRU(64, activation='tanh', return_sequences=True),
  GRU(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [32]:
loss = Huber() 
optimizer = Adam(learning_rate=0.001)
lstm_model.compile(loss=loss, optimizer=optimizer, metrics=['mse', 'mae'])
rnn_model.compile(loss=loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse', 'mae'])
gru_model.compile(loss=loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse', 'mae'])

In [33]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10)
       
es = EarlyStopping(monitor='val_loss', patience=20)

mc_lstm = ModelCheckpoint('./Control_Weights/low_neuron_group/base_data/base_multi_stne_lstm_weight_neuron.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_lstm_ns = ModelCheckpoint('./Control_Weights/low_neuron_group/base_data/base_multi_stne_lstm_weight_ns_neuron.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_rnn = ModelCheckpoint('./Control_Weights/low_neuron_group/base_data/base_multi_stne_rnn_weight_neuron.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_rnn_ns = ModelCheckpoint('./Control_Weights/low_neuron_group/base_data/base_multi_stne_rnn_weight_ns_neuron.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_gru = ModelCheckpoint('./Control_Weights/low_neuron_group/base_data/base_multi_stne_gru_weight_neuron.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_gru_ns = ModelCheckpoint('./Control_Weights/low_neuron_group/base_data/base_multi_stne_gru_weight_ns_neuron.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)

In [34]:
lstm_history = lstm_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_lstm])
lstm_history_ns = lstm_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_lstm_ns])

Epoch 1/500
   1490/Unknown - 9s 3ms/step - loss: 2.1736 - mse: 25.4537 - mae: 2.5387
Epoch 1: val_loss improved from inf to 2.16792, saving model to ./Control_Weights/low_neuron_group/base_data\base_multi_stne_lstm_weight_neuron.h5
1503/1503 [==============================] - 11s 4ms/step - loss: 2.1718 - mse: 25.4091 - mae: 2.5368 - val_loss: 2.1679 - val_mse: 21.9337 - val_mae: 2.5496 - lr: 0.0010
Epoch 2/500
1499/1503 [============================>.] - ETA: 0s - loss: 1.3162 - mse: 9.5795 - mae: 1.6609
Epoch 2: val_loss improved from 2.16792 to 1.97625, saving model to ./Control_Weights/low_neuron_group/base_data\base_multi_stne_lstm_weight_neuron.h5
1503/1503 [==============================] - 5s 4ms/step - loss: 1.3166 - mse: 9.5843 - mae: 1.6615 - val_loss: 1.9763 - val_mse: 18.4420 - val_mae: 2.3683 - lr: 0.0010
Epoch 3/500
1488/1503 [============================>.] - ETA: 0s - loss: 1.2754 - mse: 9.0462 - mae: 1.6179
Epoch 3: val_loss improved from 1.97625 to 1.92555, saving m

In [35]:
lstm_save_path = f"./Control_Models/low_neuron_group/base_data/lstm_model"

save_model(lstm_model, lstm_save_path, overwrite=True)

INFO:tensorflow:Assets written to: ./Control_Models/low_neuron_group/base_data/lstm_model\assets


INFO:tensorflow:Assets written to: ./Control_Models/low_neuron_group/base_data/lstm_model\assets


In [36]:
rnn_history = rnn_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_rnn])
rnn_history_ns = rnn_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_rnn_ns])

Epoch 1/500
   1495/Unknown - 4s 2ms/step - loss: 1.9375 - mse: 21.3509 - mae: 2.2922
Epoch 1: val_loss improved from inf to 2.10866, saving model to ./Control_Weights/low_neuron_group/base_data\base_multi_stne_rnn_weight_neuron.h5
1503/1503 [==============================] - 5s 2ms/step - loss: 1.9382 - mse: 21.3562 - mae: 2.2930 - val_loss: 2.1087 - val_mse: 21.2807 - val_mae: 2.4819 - lr: 0.0010
Epoch 2/500
1477/1503 [============================>.] - ETA: 0s - loss: 1.2833 - mse: 9.0947 - mae: 1.6277
Epoch 2: val_loss improved from 2.10866 to 1.89277, saving model to ./Control_Weights/low_neuron_group/base_data\base_multi_stne_rnn_weight_neuron.h5
1503/1503 [==============================] - 3s 2ms/step - loss: 1.2937 - mse: 9.2644 - mae: 1.6385 - val_loss: 1.8928 - val_mse: 17.9862 - val_mae: 2.2562 - lr: 0.0010
Epoch 3/500
1496/1503 [============================>.] - ETA: 0s - loss: 1.2577 - mse: 8.9496 - mae: 1.5972
Epoch 3: val_loss did not improve from 1.89277
1503/1503 [=====

In [37]:
rnn_save_path = f"./Control_Models/low_neuron_group/base_data/rnn_model"

save_model(rnn_model, rnn_save_path, overwrite=True)

INFO:tensorflow:Assets written to: ./Control_Models/low_neuron_group/base_data/rnn_model\assets


INFO:tensorflow:Assets written to: ./Control_Models/low_neuron_group/base_data/rnn_model\assets


In [38]:
gru_history = gru_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_gru])
gru_history_ns = gru_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_gru_ns])

Epoch 1/500
   1503/Unknown - 9s 4ms/step - loss: 1.9603 - mse: 21.6574 - mae: 2.3235
Epoch 1: val_loss improved from inf to 2.12611, saving model to ./Control_Weights/low_neuron_group/base_data\base_multi_stne_gru_weight_neuron.h5
1503/1503 [==============================] - 10s 4ms/step - loss: 1.9603 - mse: 21.6574 - mae: 2.3235 - val_loss: 2.1261 - val_mse: 21.0049 - val_mae: 2.5121 - lr: 0.0010
Epoch 2/500
1501/1503 [============================>.] - ETA: 0s - loss: 1.3086 - mse: 9.3471 - mae: 1.6556
Epoch 2: val_loss did not improve from 2.12611
1503/1503 [==============================] - 6s 4ms/step - loss: 1.3090 - mse: 9.3541 - mae: 1.6560 - val_loss: 2.2927 - val_mse: 24.2351 - val_mae: 2.6776 - lr: 0.0010
Epoch 3/500
1501/1503 [============================>.] - ETA: 0s - loss: 1.2593 - mse: 8.8912 - mae: 1.6016
Epoch 3: val_loss improved from 2.12611 to 1.99628, saving model to ./Control_Weights/low_neuron_group/base_data\base_multi_stne_gru_weight_neuron.h5
1503/1503 [====

In [39]:
gru_save_path = f"./Control_Models/low_neuron_group/base_data/gru_model"

save_model(gru_model, gru_save_path, overwrite=True)

INFO:tensorflow:Assets written to: ./Control_Models/low_neuron_group/base_data/gru_model\assets


INFO:tensorflow:Assets written to: ./Control_Models/low_neuron_group/base_data/gru_model\assets


In [40]:
# 종합 결과

history_list = ["lstm_history", "rnn_history", "gru_history", "lstm_history_ns", "rnn_history_ns", "gru_history_ns"]
def result(historys) :
  for name, history in globals().items() :
    if name in history_list :
      print(f"-------------{name}-------------")
      val_loss = min(history.history['val_loss'])
      val_mse = min(history.history['val_mse'])
      val_mae = min(history.history['val_mae'])
      print(f"{name} Validation Loss:", val_loss)
      print(f"{name} Validation MSE:", val_mse)
      print(f"{name} Validation MAE:", val_mae)

result(history_list)

-------------lstm_history-------------
lstm_history Validation Loss: 1.7561427354812622
lstm_history Validation MSE: 16.306522369384766
lstm_history Validation MAE: 2.1313395500183105
-------------lstm_history_ns-------------
lstm_history_ns Validation Loss: 1.803473949432373
lstm_history_ns Validation MSE: 16.808210372924805
lstm_history_ns Validation MAE: 2.166118621826172
-------------rnn_history-------------
rnn_history Validation Loss: 1.7866716384887695
rnn_history Validation MSE: 16.58176612854004
rnn_history Validation MAE: 2.1382408142089844
-------------rnn_history_ns-------------
rnn_history_ns Validation Loss: 1.9072301387786865
rnn_history_ns Validation MSE: 17.918180465698242
rnn_history_ns Validation MAE: 2.2762937545776367
-------------gru_history-------------
gru_history Validation Loss: 1.7294440269470215
gru_history Validation MSE: 15.776406288146973
gru_history Validation MAE: 2.1038787364959717
-------------gru_history_ns-------------
gru_history_ns Validation Loss